In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import time
import json

In [2]:
DRIVER_PATH = "./driver/chromedriver"

## Get US News Links of Each College

In [3]:
colleges_usnews_links = {}

with open("./college_names_id.json") as f:
    college_names = json.load(f)

In [4]:
def get_usnews_link(college_id, college_name):
    link_result = {
        "college_name": college_name,
        "usnews_link": ""
    }
    query_str = "https://www.google.com/search?q=us+news+" + college_name.replace(" ", "+")
    try:
        driver = webdriver.Chrome(executable_path=DRIVER_PATH)
        driver.get(query_str)
        
        body = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )
        try:
            first_search_result = body.find_element(By.CLASS_NAME, value="yuRUbf")
            search_link = first_search_result.find_element(By.TAG_NAME, value="a").get_attribute("href")
            link_result["usnews_link"] = search_link
        except:
            pass
    finally:
        driver.quit()
    colleges_usnews_links[college_id] = link_result

In [5]:
for key, value in college_names.items():
    get_usnews_link(key, value)

/Users/minzhou/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [6]:
with open("./colleges_usnews_link.json", "w") as outfile:
    outfile.write(json.dumps(colleges_usnews_links, indent=4))

## Filter invalid links

In [33]:
updated_colleges_usnews_links = {}

In [34]:
with open("./colleges_usnews_link.json") as f:
    all_colleges_usnews_links = json.load(f)

In [35]:
# Clean all the urls without "best colleges"
for key, value in all_colleges_usnews_links.items():
    if "best-colleges" in value["usnews_link"] and len(value["usnews_link"]) > 36:
        cur_link_split = value["usnews_link"].split("/")
        if len(cur_link_split) >= 5:
            purified_link = '/'.join(cur_link_split[0:5]).split("#")[0]
            if purified_link[-4:].isdigit():
                updated_colleges_usnews_links[key] = value
                updated_colleges_usnews_links[key]["usnews_link"] = purified_link

In [36]:
len(updated_colleges_usnews_links.items())

1551

In [37]:
with open("./colleges_usnews_link_updated.json", "w") as outfile:
    outfile.write(json.dumps(updated_colleges_usnews_links, indent=4))

In [39]:
# group by same link
# get last 4 digits of each usnews link, then group
# After that, manually clean up ids not match
usnews_ids = {}

In [40]:
for key, value in updated_colleges_usnews_links.items():
    cur_id = value["usnews_link"][-4:]
    if cur_id not in usnews_ids.keys():
        usnews_ids[cur_id] = []
    usnews_ids[cur_id].append(key)

In [ ]:
for key, value in usnews_ids.items():
    if len(value) > 1:
        print(key)